# Etapa 3 — Automação Profissional do Processo

Nesta etapa, o processo de extração, transformação e exportação de dados do e-commerce é automatizado de forma modular e reprodutível, seguindo boas práticas de desenvolvimento em Python (PEP 8) e registro em log.

---


In [7]:
# ==============================================================
# E-commerce Data Analytics - Automação de Processos
# Autor: Gustavo de Paula Silva
# Descrição:
#   Este script realiza a extração, transformação, análise e
#   exportação automatizada de dados de um e-commerce fictício,
#   utilizando Python, SQLite e Pandas.
# ==============================================================

import pandas as pd
import sqlite3
import os
import logging
import pathlib
from datetime import datetime

# -------------------- CONFIGURAÇÕES INTELIGENTES --------------------
if "__file__" in globals():
    ROOT_DIR = pathlib.Path(__file__).resolve().parent
else:
    ROOT_DIR = pathlib.Path().resolve().parent

DB_PATH = ROOT_DIR / "data" / "ecommerce_realista.db"
EXPORT_PATH = ROOT_DIR / "data_export"
os.makedirs(EXPORT_PATH, exist_ok=True)

logging.basicConfig(
    filename=f'{EXPORT_PATH}/log_execucao.txt',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)


# -------------------- FUNÇÕES --------------------
def carregar_dados(db_path: str):
    """
    Conecta ao banco SQLite e carrega as tabelas principais.

    Parâmetros
    ----------
    db_path : str
        Caminho do arquivo .db que contém as tabelas `clientes`,
        `produtos` e `vendas`.

    Retorna
    -------
    tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]
        Três DataFrames correspondentes às tabelas:
        clientes, produtos e vendas.

    Exceções
    --------
    FileNotFoundError
        Caso o arquivo .db não seja encontrado.
    sqlite3.Error
        Caso ocorra algum erro de leitura no banco de dados.
    """
    try:
        with sqlite3.connect(db_path) as conn:
            clientes = pd.read_sql("SELECT * FROM clientes", conn)
            produtos = pd.read_sql("SELECT * FROM produtos", conn)
            vendas = pd.read_sql("SELECT * FROM vendas", conn)
        logging.info("Tabelas carregadas com sucesso.")
        return clientes, produtos, vendas
    except FileNotFoundError:
        logging.error(f"Banco de dados não encontrado em {db_path}")
        raise
    except sqlite3.Error as e:
        logging.error(f"Erro ao carregar tabelas: {e}")
        raise


def consolidar_dados(clientes: pd.DataFrame, produtos: pd.DataFrame, vendas: pd.DataFrame) -> pd.DataFrame:
    """
    Consolida as tabelas `vendas`, `produtos` e `clientes` em um único DataFrame.

    Parâmetros
    ----------
    clientes : pd.DataFrame
        Tabela contendo informações de clientes.
    produtos : pd.DataFrame
        Tabela contendo informações de produtos.
    vendas : pd.DataFrame
        Tabela contendo as vendas realizadas.

    Retorna
    -------
    pd.DataFrame
        DataFrame unificado com todas as informações e colunas adicionais
        de data formatada e período (`ano_mes`).

    Exceções
    --------
    KeyError
        Caso as colunas esperadas ('id_produto', 'id_cliente', 'data_venda')
        não existam em alguma tabela.
    """
    try:
        df = (
            vendas.merge(produtos, on='id_produto', how='left')
                  .merge(clientes, on='id_cliente', how='left')
        )
        df['data_venda'] = pd.to_datetime(df['data_venda'])
        df['ano_mes'] = df['data_venda'].dt.to_period('M')
        logging.info("Dados consolidados com sucesso.")
        return df
    except KeyError as e:
        logging.error(f"Erro de coluna ao consolidar dados: {e}")
        raise


def gerar_resumos(df: pd.DataFrame):
    """
    Gera análises resumidas e o ticket médio do e-commerce.

    Parâmetros
    ----------
    df : pd.DataFrame
        DataFrame consolidado com informações de vendas, clientes e produtos.

    Retorna
    -------
    tuple[pd.DataFrame, pd.DataFrame, float]
        - resumo_cat : faturamento total por categoria de produto
        - resumo_cli : faturamento total por cliente
        - ticket_medio : valor médio das vendas (float)

    Notas
    -----
    Esta função é usada para gerar indicadores de negócio, como:
    - categorias que mais faturam
    - clientes mais valiosos
    - desempenho médio de venda
    """
    resumo_cat = df.groupby('categoria', as_index=False)['valor_total'].sum()
    resumo_cat.rename(columns={'valor_total': 'faturamento_total'}, inplace=True)

    resumo_cli = df.groupby('nome', as_index=False)['valor_total'].sum()
    resumo_cli.rename(columns={'valor_total': 'faturamento_cliente'}, inplace=True)

    ticket_medio = df['valor_total'].mean()
    logging.info(f"Ticket médio calculado: R$ {ticket_medio:.2f}")
    return resumo_cat, resumo_cli, ticket_medio


def exportar_arquivos(df: pd.DataFrame, resumo_cat: pd.DataFrame, resumo_cli: pd.DataFrame):
    """
    Exporta os DataFrames gerados para arquivos .csv e registra a execução.

    Parâmetros
    ----------
    df : pd.DataFrame
        DataFrame consolidado com todas as vendas.
    resumo_cat : pd.DataFrame
        Resumo de faturamento por categoria.
    resumo_cli : pd.DataFrame
        Resumo de faturamento por cliente.

    Retorna
    -------
    None
        Exporta os arquivos para a pasta `data_export`.

    Exceções
    --------
    PermissionError
        Caso algum arquivo CSV esteja aberto no momento da exportação.
    """
    data_exec = datetime.now().strftime('%Y-%m-%d')
    try:
        df.to_csv(f'{EXPORT_PATH}/vendas_detalhadas_{data_exec}.csv', index=False)
        resumo_cat.to_csv(f'{EXPORT_PATH}/resumo_categorias_{data_exec}.csv', index=False)
        resumo_cli.to_csv(f'{EXPORT_PATH}/resumo_clientes_{data_exec}.csv', index=False)
        logging.info("Arquivos CSV exportados com sucesso.")
    except PermissionError:
        logging.error("Erro: Feche os arquivos CSV antes de exportar novamente.")
        raise
    except Exception as e:
        logging.error(f"Erro ao exportar arquivos: {e}")
        raise


def main():
    """
    Função principal que coordena a execução do pipeline de automação.

    Etapas
    ------
    1. Verifica a existência do banco de dados.
    2. Carrega as tabelas principais.
    3. Consolida os dados em um único DataFrame.
    4. Gera análises resumidas e calcula o ticket médio.
    5. Exporta os resultados e atualiza o log de execução.

    Retorna
    -------
    None
        Exibe no terminal mensagens de progresso e sucesso.
    """
    print("🚀 Iniciando automação de dados...\n")
    if not os.path.exists(DB_PATH):
        raise FileNotFoundError("Banco de dados não encontrado.")
    clientes, produtos, vendas = carregar_dados(DB_PATH)
    df = consolidar_dados(clientes, produtos, vendas)
    resumo_cat, resumo_cli, ticket_medio = gerar_resumos(df)
    exportar_arquivos(df, resumo_cat, resumo_cli)
    print(f"✅ Processo concluído com sucesso! Ticket médio: R$ {ticket_medio:.2f}")


if __name__ == "__main__":
    main()

🚀 Iniciando automação de dados...

✅ Processo concluído com sucesso! Ticket médio: R$ 391.43


### Resultados Esperados

- Conexão bem-sucedida ao banco `ecommerce_realista.db`
- Geração dos arquivos:
  - `vendas_detalhadas_<data>.csv`
  - `resumo_categorias_<data>.csv`
  - `resumo_clientes_<data>.csv`
- Atualização automática do arquivo `log_execucao.txt` com data e hora.
- Exibição do ticket médio e confirmação de sucesso da automação.